In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
iOptions = [2,2,5,9,10,16,15]
keffs = [1.0,1.0,1.0,0.99,0.98,0.995,0.953]
centralities = ["0%-93%"]
Opt_names = ["no ML eID","ML eID", "conv partner","tighter match","conv pattern", "ML conv rej", "ML hit, ddphi"]

In [3]:
N_centr = 6
N_options = 6
N_pt = 1
colors0=[1,2,root.kGreen+2,4,root.kCyan,root.kMagenta,root.kOrange+4,root.kGray,root.kGray,root.kBlack,root.kYellow,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
colors=[2,4,4,root.kGreen+2,root.kOrange+4,root.kMagenta,root.kGray,root.kGray,root.kBlack,root.kCyan,root.kOrange,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="../dca/input/data/"
file_names=["m_ee_DCA_531.root","m_ee_DCA_531.root","m_ee_DCA_532.root","m_ee_DCA_533.root","m_ee_DCA_59.root","m_ee_DCA_537.root","m_ee_DCA_803.root"]#,"m_ee_DCA_535.root"],"m_ee_DCA_536.root"]
hist_names = [["inv_mass_stat_00","inv_mass_syst_00",f"inv_mass_ee_DCA_V{iOption}_FG12",f"inv_mass_ee_DCA_V{iOption}_BG12"] for iOption in iOptions]
N_type = len(hist_names[0])

In [4]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hists_emc_read0.append(infile.Get(hist_names[iFile][itype]))
        try: hists_emc_read0[-1].SetDirectory(root.nullptr); hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"_{iFile}")
        except: 
            print(f"Error setting directory for {hist_names[iFile][itype]}{0} in file {file_names[iFile]}")
            break
        hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"{iFile}")
    hists_read.append(hists_emc_read0)

    infile.Close()


Error setting directory for inv_mass_ee_DCA_V10_FG120 in file m_ee_DCA_59.root
Error setting directory for inv_mass_ee_DCA_V15_FG120 in file m_ee_DCA_803.root


In [5]:
infile = root.TFile.Open("../dca/input/Star.root", "READ")

d = infile.Get("Table 1")                  # TDirectoryFile
hists_star = d.Get("Graph1D_y1")           # TGraphAsymmErrors

# Optional safety check
if not hists_star:
    raise RuntimeError("Graph not found")

hists_star.SetName("STAR_AuAu")             # optional
hists_star.SetTitle("STAR Au+Au 0–80%")

infile.Close()

In [6]:

# ---- extract x values once ----
n = hists_star.GetN()
x = hists_star.GetX()
#  ---- set x errors from neighbor spacing ----
for i in range(n):
    if i == 0:
        dx = 0.5 * (x[1] - x[0])
    elif i == n - 1:
        dx = 0.5 * (x[n-1] - x[n-2])
    else:
        dx = 0.5 * (x[i+1] - x[i-1])

    hists_star.SetPointEXlow(i,  dx/2)
    hists_star.SetPointEXhigh(i, dx/2)


In [7]:
import ROOT as root

f = root.TFile.Open("../dca/input/Star200.root","READ")
d = f.Get("Table 13")

#d.ls()  # <-- shows the real object names in that directory

g = d.Get("Graph2D_y1;1")   # NOTE the ;1 (often required)
if not g:
    g = d.Get("Graph2D_y1") # fallback
if not g:
    raise RuntimeError("Can't find Graph2D_y1 in Table 13")

#c = root.TCanvas("c","c",900,650)
#g.Draw("P0")     # for TGraph2D-style scatter (what you see in your screenshot)
#c.Draw()         # important for notebook display


In [8]:
in_file = root.TFile.Open("output/sim_cocktail_massdep_corr3.root","read")
in_file.ls()
sum_sim_proj_copycat = in_file.Get("sum_sim_proj_copycat;1")
sum_sim_proj_copycat.SetDirectory(root.nullptr)
in_file.Close()

TFile**		output/sim_cocktail_massdep_corr3.root	
 TFile*		output/sim_cocktail_massdep_corr3.root	
  KEY: TH1D	sum_sim_proj_copycat;1	


In [9]:
#infile_phenix_recoeff = root.TFile.Open("../syst/output/phenix_cock_recoeff.root","READ")
#phenix_star_reco_eff = infile_phenix_recoeff.Get("ph_mass_05_hist_reb")
#phenix_star_reco_eff.SetDirectory(root.nullptr)
#infile_phenix_recoeff.Close()

In [10]:
accepance_eff_low_mass = root.TF1("accepance_eff_low_mass","pol2",0,0.7)
accepance_eff_low_mass.SetParameters(2.7249856802644163e-05 , -7.603255066272503e-05 , 9.481844932778539e-05)
accepance_eff_high_mass = root.TF1("accepance_eff_high_mass","pol2",0.7,5.0 )
accepance_eff_high_mass.SetParameters(2.7249856802644163e-05 , -7.603255066272503e-05 , 9.481844932778539e-05)

for ibin in range(1,hists_read[len(file_names)-1][0].GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    mass = hists_read[len(file_names)-1][0].GetBinCenter(ibin)
    #if mass < 0.6:
    #    eff = accepance_eff_low_mass.Eval(0.6)
    #elif mass < 2.9:
    #    eff = accepance_eff_high_mass.Eval(mass)
    #else: eff = accepance_eff_high_mass.Eval(mass)
    eff = sum_sim_proj_copycat.GetBinContent(sum_sim_proj_copycat.FindBin(mass))
    if mass>3.5: eff = sum_sim_proj_copycat.GetBinContent(sum_sim_proj_copycat.FindBin(3.4))
    #eff /= phenix_star_reco_eff.GetBinContent(phenix_star_reco_eff.FindBin(mass))+1e-32
    #print(f"mass: {mass}, eff: {eff}")
    if eff > 0:
        hists_read[len(file_names)-1][0].SetBinContent(ibin, hists_read[len(file_names)-1][0].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][0].SetBinError(ibin, hists_read[len(file_names)-1][0].GetBinError(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinContent(ibin, hists_read[len(file_names)-1][1].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinError(ibin, hists_read[len(file_names)-1][1].GetBinError(ibin)/eff)
    

In [11]:
dnch_phenix = 650
#dnch_phenix = 1
dnch_star  = 180*2
dnch_star200 = 750*2
ncoll_phenix = 260
ncoll_star200 = 350
iOpt = len(file_names)-1
hists_star.Scale(1./dnch_star)
hists_read[iOpt][0].Scale(1./dnch_phenix)
hists_read[iOpt][1].Scale(1./dnch_phenix)

In [12]:
file_thermal = root.TFile.Open("../syst/output/hists/thermal3.root","READ")
hist_names = ["h2_m_pt","h2_m_pt_mother05","h2_m_pt05","h2_m_pt035","h2_m_pt_nocut"]
hists_thermal = []
for ihist in range(len(hist_names)):
    hists_thermal.append(file_thermal.Get(hist_names[ihist]))
    hists_thermal[-1].SetDirectory(root.nullptr)
file_thermal.Close()                         

In [13]:
star_reco_eff = hists_thermal[0].ProjectionY(hists_thermal[0].GetName()+"_reco_eff")
star_reco_eff.Divide(hists_thermal[-1].ProjectionY(hists_thermal[-1].GetName()+"_reco_eff"))
star_reco_eff.Smooth(10)
phenix_reco_eff = hists_thermal[1].ProjectionY(hists_thermal[1].GetName()+"_reco_eff")
phenix_reco_eff.Divide(hists_thermal[2].ProjectionY(hists_thermal[2].GetName()+"_reco_eff"))
phenix_reco_eff.Smooth(10)

In [14]:
#for ibin in range(1,hists_read[len(file_names)-1][0].GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
#    mass = hists_read[len(file_names)-1][0].GetBinCenter(ibin)
#    eff = 1
#    if phenix_reco_eff.GetBinContent(phenix_reco_eff.FindBin(mass)) > 0:
#        eff = 1./phenix_reco_eff.GetBinContent(phenix_reco_eff.FindBin(mass))
#    if eff > 0:
#        hists_read[len(file_names)-1][0].SetBinContent(ibin, hists_read[len(file_names)-1][0].GetBinContent(ibin)/eff)
#        hists_read[len(file_names)-1][0].SetBinError(ibin, hists_read[len(file_names)-1][0].GetBinError(ibin)/eff)
#        hists_read[len(file_names)-1][1].SetBinContent(ibin, hists_read[len(file_names)-1][1].GetBinContent(ibin)/eff)
#        hists_read[len(file_names)-1][1].SetBinError(ibin, hists_read[len(file_names)-1][1].GetBinError(ibin)/eff)

In [15]:
infile_star_recoeff = root.TFile.Open("../syst/output/star_cock_recoeff.root","READ")
star_reco_eff = infile_star_recoeff.Get("mass_05_hist_reb")
star_reco_eff.SetDirectory(root.nullptr)
infile_star_recoeff.Close()

In [16]:
f_200 = root.TFile.Open("../dca/input/Star200.root","READ")
g200 = f_200.Get("Table 13").Get("Graph2D_y1;1")
if not g200:
    g200 = f_200.Get("Table 13").Get("Graph2D_y1")
if not g200:
    raise RuntimeError("Graph2D_y1 not found")

n = g200.GetN()
f_200.Close

gr_200 = root.TGraphErrors(n)
for i in range(n):
    m  = float(g200.GetX()[i])  # mass
    acc = 1./star_reco_eff.GetBinContent(star_reco_eff.FindBin(min(m,3)))+1e-28 
    #acc = acc/(1.-m/4.)
    dm = float(g200.GetY()[i])  # dmass (usually bin width)
    val= float(g200.GetZ()[i]/dnch_phenix*0.75)  # dN/dmass
    val_err = float(g200.GetErrorZ(i)/dnch_phenix*0.75)  # error on dN/dmass
    gr_200.SetPoint(i, m, val/acc)
    gr_200.SetPointError(i, 0.5*dm, val_err/acc)  # x-error from bin width



In [17]:
c = root.TCanvas("c2","c2",900,650)
root.gPad.SetLogy()
gr_200.SetMarkerStyle(20)
gr_200.SetMarkerColor(root.kGreen+2)
gr_200.SetLineColor(root.kGreen+2)
gr_200.SetTitle("Table 13; mass; dN/dmass")
gr_200.Draw("AP")
c.Draw()

In [18]:
sim_file_path="../sim/output/spring26/dca/"
sim_file_names=['pi0_gee_v00.root', 'phi_ee_v00.root', "phi_ee_v00.root", 'jpsi_ee_v01.root', 'ccbar_soft_v00.root', 'ccbar_soft_v00.root',\
                 'omega_ee_v00.root', 'ccbar_soft_v00.root', 'thermal_v00.root',  'ccbar_soft_v00.root', 'ccbar_soft_v00.root','ccbar_soft_v00.root','ccbar_soft_v00.root', 'ccbar_soft_v00.root']#thermal_25M_200_v0

sim_file_names=['pi0_gee_v01.root','omega_ee_v01.root', 'phi_ee_v01.root', 'jpsi_ee_v01.root']#thermal_25M_200_v0
hist_select_3D_names = ["inv_mass_dca_gen_5","inv_mass_dca_gen_6","inv_mass_dca_gen_7","inv_mass_dca_gen_8","inv_mass_dca_gen_9","inv_mass_dca_fg4_5","inv_mass_dca_fg4_6","inv_mass_dca_fg4_7","inv_mass_dca_fg4_8","inv_mass_dca_fg4_9"]
hists_sim_phenix_recoeff = []
for iFile in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_phenix_recoeff0 = []
    for ihist in range(len(hist_select_3D_names)):
        hist = infile.Get(hist_select_3D_names[ihist])
        if not hist:
            print(f"Error: histogram {hist_select_3D_names[ihist]} not found in file {sim_file_names[iFile]}")
            continue
        hist.SetDirectory(root.nullptr)
        hist.SetName(hist.GetName()+f"_{iFile}")
        hists_sim_phenix_recoeff0.append(hist)
    hists_sim_phenix_recoeff.append(hists_sim_phenix_recoeff0)
    infile.Close()

In [19]:
n_entries = [0]*len(sim_file_names)
for iFile in range(len(sim_file_names)):
    for ihist in range(0,len(hists_sim_phenix_recoeff[iFile])):
        n_entries[iFile] += hists_sim_phenix_recoeff[iFile][ihist].GetEntries()
        hists_sim_phenix_recoeff[iFile][ihist] = hists_sim_phenix_recoeff[iFile][ihist].ProjectionY(hists_sim_phenix_recoeff[iFile][0].GetName()+"_proj")

In [20]:
for iFile in range(len(sim_file_names)):
    for ihist in range(1,5):
        hists_sim_phenix_recoeff[iFile][0].Add(hists_sim_phenix_recoeff[iFile][ihist])
        hists_sim_phenix_recoeff[iFile][5].Add(hists_sim_phenix_recoeff[iFile][ihist+5])

In [21]:
for iFile in range(len(sim_file_names)):
    hists_sim_phenix_recoeff[iFile][5].Scale(hists_sim_phenix_recoeff[iFile][0].Integral()/hists_sim_phenix_recoeff[iFile][5].Integral()*hists_sim_phenix_recoeff[iFile][0].GetBinWidth(1)/hists_sim_phenix_recoeff[iFile][5].GetBinWidth(1))

In [22]:
for iFile in range(len(sim_file_names)):
    hists_sim_phenix_recoeff[iFile][5].Scale(10.0 / hists_sim_phenix_recoeff[iFile][0].GetBinWidth(1)/n_entries[iFile]/ dnch_phenix)

In [23]:
file_names_star_cock = ["star_pi0_gee.root", "star_omega_ee.root","star_phi_ee.root", "star_jpsi_ee.root"]
cock_hist_in_phenix = []; n_entries = []
for iFile, fname in enumerate(file_names_star_cock):
    f = root.TFile.Open("../syst/output/hists/" + fname)
    name = "h2_m_pt_mother05"
    cock_hist_in_phenix.append(f.Get(name))
    cock_hist_in_phenix[-1].SetDirectory(root.nullptr)
    cock_hist_in_phenix[-1].SetName("h2_m_pt_mother05" + f"_{iFile}")
    n_entries.append(f.Get("h_mult_final").GetEntries())
    f.Close()

In [24]:
print(n_entries,cock_hist_in_phenix[0].GetEntries())

[1000000.0, 1000000.0, 1000000.0, 1000000.0] 499142.0


In [25]:
star_mass_proj = []
for iFile in range(len(cock_hist_in_phenix)):
    star_mass_proj.append(cock_hist_in_phenix[iFile].ProjectionY("mass_proj_"+str(iFile)))
    star_mass_proj[iFile].Scale(1./ n_entries[iFile] / star_mass_proj[-1].GetBinWidth(1) * 2 / dnch_phenix)

In [26]:
mass_res = array("d", [0.01,0.37,1.019,2.6]); mass_res_ratio = array("d",[])
for iFile in range(len(sim_file_names)):
    print(f"File {iFile}: {hists_sim_phenix_recoeff[iFile][5].Integral():.2g}")
for iFile in range(len(cock_hist_in_phenix)):
    print(f"File {iFile}: {star_mass_proj[iFile].Integral():.2g},  {hists_sim_phenix_recoeff[iFile][5].Integral()/star_mass_proj[iFile].Integral()/star_mass_proj[iFile].GetBinWidth(1)*hists_sim_phenix_recoeff[iFile][5].GetBinWidth(1):.2g}")
    mass_res_ratio.append(hists_sim_phenix_recoeff[iFile][5].Integral()/star_mass_proj[iFile].Integral()/star_mass_proj[iFile].GetBinWidth(1)*hists_sim_phenix_recoeff[iFile][5].GetBinWidth(1))
    star_mass_proj[iFile].Scale(hists_sim_phenix_recoeff[iFile][5].Integral()/star_mass_proj[iFile].Integral()/star_mass_proj[iFile].GetBinWidth(1)*hists_sim_phenix_recoeff[iFile][5].GetBinWidth(1))

graph_ratio_mass = root.TGraph(4,mass_res,mass_res_ratio)
c6 = root.TCanvas("c6","c6",600,600)
Format_Hist_total(graph_ratio_mass)
graph_ratio_mass.Draw("AP")
c6.Draw()

File 0: 0.016
File 1: 3.1e-05
File 2: 3.3e-05
File 3: 5.4e-06
File 0: 0.33,  0.24
File 1: 0.00012,  1.3
File 2: 0.00012,  1.4
File 3: 3.8e-05,  0.72


In [27]:
# Rescale gr_200 points using linear interpolation/extrapolation from graph_ratio_mass

def eval_tgraph_linear_extrap(gr, xval):
    xs = [float(gr.GetX()[i]) for i in range(gr.GetN())]
    ys = [float(gr.GetY()[i]) for i in range(gr.GetN())]

    # ensure points are ordered by x
    pts = sorted(zip(xs, ys), key=lambda p: p[0])
    xs, ys = zip(*pts)

    if xval <= xs[0]:
        x1, y1 = xs[0], ys[0]
        x2, y2 = xs[1], ys[1]
    elif xval >= xs[-1]:
        x1, y1 = xs[-2], ys[-2]
        x2, y2 = xs[-1], ys[-1]
    else:
        for j in range(len(xs) - 1):
            if xs[j] <= xval <= xs[j + 1]:
                x1, y1 = xs[j], ys[j]
                x2, y2 = xs[j + 1], ys[j + 1]
                break

    if x2 == x1:
        return float(y1)
    return float(y1 + (y2 - y1) * (xval - x1) / (x2 - x1))


for i in range(gr_200.GetN()):
    x = float(gr_200.GetX()[i])
    y = float(gr_200.GetY()[i])
    ex = float(gr_200.GetErrorX(i))
    ey = float(gr_200.GetErrorY(i))

    scale = eval_tgraph_linear_extrap(graph_ratio_mass, x)

    gr_200.SetPoint(i, x, y * scale)
    gr_200.SetPointError(i, ex, ey * scale)

In [34]:
canvases = []
legends =[]
for ipar in [0]:#[0,1,2,3,4,5]:
    c0 = root.TCanvas(f"c0_{ipar}",f"c0_{ipar}",1100,1100)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = len(file_names)-1
    rebin = 1

    centr_length = [1,0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
    for icentr in range(ipar,ipar+1):
        legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[icentr]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()
        c0.cd(1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0],"m_{e^{+}e^{-}} [GeV]","d^{2}N/dydm_{e^{+}e^{-}}#divide dN_{ch}/d#eta|_{#eta=0} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,1,3,1,0.99,1.35)
        hists_read[iOpt][1].SetFillColorAlpha(1,0.4)
        hists_read[iOpt][0].Draw("")
        hists_read[iOpt][1].Draw("same e2")
        legends.append(root.TLegend(0.25,0.83,0.99,0.89,"p_{T}^{e }>0.3 GeV, |y^{e}|<0.35"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()      

        #sum_sim_proj.Draw("same L")
        hists_read[iOpt][0].SetMaximum(2e-1*650/dnch_phenix)
        hists_read[iOpt][0].SetMinimum(5e-10*650/dnch_phenix)

        #hbd_paper.SetMaximum(2e-1)
        gr_200.Draw("same P L")


        hists_read[iOpt][0].Draw("same")
        hists_read[iOpt][1].Draw("same e2")



        legends.append(root.TLegend(0.3,0.65,0.95,0.9))
        Format_Legend(legends[-1],0.04,12)
        legends[-1].AddEntry(0,"","")
        legends[-1].SetNColumns(2)
        legends[-1].AddEntry(0,"","")
        legends[-1].AddEntry(hists_read[iOpt][0],"Mine","pl")
        legends[-1].AddEntry(gr_200,"STAR 200 GeV","pl")
        #legends[-1].AddEntry(hists_sim_porj_ccbar,"c#bar{c} simulation","l")
        #legends[-1].AddEntry(hbd_paper_ccbar_hist,"c#bar{c} from HBD PHENIX","l")
        #legends[-1].AddEntry(hist_ccbar_star,"c#bar{c} from STAR 27 GeV","l")
        #legends[-1].AddEntry(star200_paper_ccbar_hist,"c#bar{c} from STAR 200 GeV","l")
        #legends[-1].AddEntry(hists_star,"#splitline{STAR #sqrt{s_{NN}} = 27 GeV}{0-80% Au+Au}","pl")
        #legends[-1].AddEntry(hists_star,"STAR #sqrt{s_{NN}} = 27 GeV 0-80% Au+Au","pl")
        legends[-1].Draw()

        #for iFile in range(len(sim_file_names)):
        #    Format_Graph(hists_sim_phenix_recoeff[iFile][0],Mstyle=21, Mcolor=colors[iFile], Lcolor=colors[iFile])
        #    hists_sim_phenix_recoeff[iFile][5].Draw("same H")
        #for iFile in range(len(star_mass_proj)):
        #    Format_Graph(star_mass_proj[iFile],Mstyle=21, Mcolor=colors[iFile], Lcolor=colors[iFile])
        #    star_mass_proj[iFile].Draw("same H")

    #if True: root.PHENIXTools.DrawPreliminary(0.6, 0.6, 0.3)


    # Draw the logo on the canvas with specified coordinates and size
    #logo.DrawImage(0.1, 0.8, 0.3, 0.9)  # Adjust the coordinates and size as needed

    c0.Draw()
    c0.Print(f"output/mass/all_corr_mass.png")

In [29]:
# Rebin gr_200 into the binning of hists_read[iOpt][0] using bin-overlap weighting
iOpt = len(file_names) - 1
h_ref = hists_read[iOpt][0]

# Target bin edges from reference histogram
nb = h_ref.GetNbinsX()
edges = array("d", [h_ref.GetBinLowEdge(1 + i) for i in range(nb)] + [h_ref.GetBinLowEdge(nb) + h_ref.GetBinWidth(nb)])

# Accumulators for integral and error^2 integral
sum_int = [0.0] * nb
sum_err2_int = [0.0] * nb

for i in range(gr_200.GetN()):
    x = float(gr_200.GetX()[i])
    y = float(gr_200.GetY()[i])
    ex = float(gr_200.GetErrorX(i))
    ey = float(gr_200.GetErrorY(i))

    # Graph "bin" range (piecewise constant)
    xl = x - ex
    xh = x + ex
    w = max(xh - xl, 0.0)
    if w <= 0:
        continue

    for b in range(nb):
        bl = edges[b]
        bh = edges[b + 1]
        ov = min(xh, bh) - max(xl, bl)  # overlap length
        if ov <= 0:
            continue

        # Integrate y over overlap; propagate error on integral
        sum_int[b] += y * ov
        sum_err2_int[b] += (ey * ov) ** 2

# Build rebinned graph at reference bin centers
gr_200_rebinned = root.TGraphErrors(nb)
gr_200_rebinned.SetName("gr_200_rebinned")
gr_200_rebinned.SetTitle("gr_200 rebinned to hists_read[iOpt][0] binning")

for b in range(nb):
    xc = h_ref.GetBinCenter(b + 1)
    bw = h_ref.GetBinWidth(b + 1)

    yb = sum_int[b] / bw if bw > 0 else 0.0
    eyb = (sum_err2_int[b] ** 0.5) / bw if bw > 0 else 0.0

    gr_200_rebinned.SetPoint(b, xc, yb)
    gr_200_rebinned.SetPointError(b, 0.5 * bw, eyb)


In [30]:

# Optional styling
gr_200_rebinned.SetMarkerStyle(24)
gr_200_rebinned.SetMarkerColor(root.kRed + 1)
gr_200_rebinned.SetLineColor(root.kRed + 1)

# Optional quick check plot
c_rebin = root.TCanvas("c_rebin", "c_rebin", 900, 650)
root.gPad.SetLogy()
h_ref.Draw("HIST")
gr_200.Draw("P SAME")
gr_200_rebinned.Draw("P SAME")
c_rebin.Draw()

In [31]:
canvases = []
legends =[]
for ipar in [0]:#[0,1,2,3,4,5]:
    c0 = root.TCanvas(f"c0_{ipar}",f"c0_{ipar}",1100,1100)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = len(file_names)-1
    rebin = 1

    centr_length = [1,0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
    for icentr in range(ipar,ipar+1):
        legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[icentr]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()
        c0.cd(1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0],"m_{e^{+}e^{-}} [GeV]","d^{2}N/dydm_{e^{+}e^{-}}#divide dN_{ch}/d#eta|_{#eta=0} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,1,3,1,0.99,1.35)
        hists_read[iOpt][1].SetFillColorAlpha(1,0.4)
        hists_read[iOpt][0].Draw("")
        hists_read[iOpt][1].Draw("same e2")
        legends.append(root.TLegend(0.25,0.83,0.99,0.89,"p_{T}^{e }>0.3 GeV, |y^{e}|<0.35"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()      

        #sum_sim_proj.Draw("same L")
        hists_read[iOpt][0].SetMaximum(2e-1)
        hists_read[iOpt][0].SetMinimum(5e-10)

        #hbd_paper.SetMaximum(2e-1)
        gr_200_rebinned.Draw("same P L")


        hists_read[iOpt][0].Draw("same")
        hists_read[iOpt][1].Draw("same e2")



        legends.append(root.TLegend(0.3,0.65,0.95,0.9))
        Format_Legend(legends[-1],0.04,12)
        legends[-1].AddEntry(0,"","")
        legends[-1].SetNColumns(2)
        legends[-1].AddEntry(0,"","")
        legends[-1].AddEntry(hists_read[iOpt][0],"Mine","pl")
        legends[-1].AddEntry(gr_200_rebinned,"STAR 200 GeV","pl")
        #legends[-1].AddEntry(hists_sim_porj_ccbar,"c#bar{c} simulation","l")
        #legends[-1].AddEntry(hbd_paper_ccbar_hist,"c#bar{c} from HBD PHENIX","l")
        #legends[-1].AddEntry(hist_ccbar_star,"c#bar{c} from STAR 27 GeV","l")
        #legends[-1].AddEntry(star200_paper_ccbar_hist,"c#bar{c} from STAR 200 GeV","l")
        #legends[-1].AddEntry(hists_star,"#splitline{STAR #sqrt{s_{NN}} = 27 GeV}{0-80% Au+Au}","pl")
        #legends[-1].AddEntry(hists_star,"STAR #sqrt{s_{NN}} = 27 GeV 0-80% Au+Au","pl")
        legends[-1].Draw()

        #for iFile in range(len(sim_file_names)):
        #    Format_Graph(hists_sim_phenix_recoeff[iFile][0],Mstyle=21, Mcolor=colors[iFile], Lcolor=colors[iFile])
        #    hists_sim_phenix_recoeff[iFile][5].Draw("same H")
        #for iFile in range(len(star_mass_proj)):
        #    Format_Graph(star_mass_proj[iFile],Mstyle=21, Mcolor=colors[iFile], Lcolor=colors[iFile])
        #    star_mass_proj[iFile].Draw("same H")

    #if True: root.PHENIXTools.DrawPreliminary(0.6, 0.6, 0.3)


    # Draw the logo on the canvas with specified coordinates and size
    #logo.DrawImage(0.1, 0.8, 0.3, 0.9)  # Adjust the coordinates and size as needed

    c0.Draw()
    c0.Print(f"output/mass/all_corr_mass.png")

In [32]:
# Convert gr_200_rebinned (TGraphErrors) to TH1 using h_ref binning
hist_200_rebinned = h_ref.Clone("hist_200_rebinned")
hist_200_rebinned.Reset("ICESM")
hist_200_rebinned.SetTitle("STAR 200 GeV rebinned;mass;dN/dmass")

for i in range(gr_200_rebinned.GetN()):
    x = float(gr_200_rebinned.GetX()[i])
    y = float(gr_200_rebinned.GetY()[i])
    ey = float(gr_200_rebinned.GetErrorY(i))
    b = hist_200_rebinned.FindBin(x)
    hist_200_rebinned.SetBinContent(b, y)
    hist_200_rebinned.SetBinError(b, ey)

# optional style
hist_200_rebinned.SetLineColor(root.kRed + 1)
hist_200_rebinned.SetMarkerColor(root.kRed + 1)
hist_200_rebinned.SetMarkerStyle(24)

In [33]:
print(hist_200_rebinned.Integral(hist_200_rebinned.FindBin(2.3), hist_200_rebinned.FindBin(3.2)))
print(hists_read[iOpt][0].Integral(hists_read[iOpt][0].FindBin(2.3), hists_read[iOpt][0].FindBin(3.2)))
print(hist_200_rebinned.Integral(hist_200_rebinned.FindBin(.0), hist_200_rebinned.FindBin(0.1)))
print(hists_read[iOpt][0].Integral(hists_read[iOpt][0].FindBin(0.0), hists_read[iOpt][0].FindBin(0.1)))
print(hist_200_rebinned.Integral(hist_200_rebinned.FindBin(0.9), hist_200_rebinned.FindBin(1.1)))
print(hists_read[iOpt][0].Integral(hists_read[iOpt][0].FindBin(0.9), hists_read[iOpt][0].FindBin(1.1)))

1.979986015391364e-06
1.946943904688938e-06
0.017610499860357166
0.014260177616703604
3.3223757033196937e-05
3.1763381470448546e-05
